# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
Arxiv has 72 new papers today
          1 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.09657


extracting tarball to tmp_2401.09657...

 done.


Found 102 bibliographic references in tmp_2401.09657/Two_massive_warm_Jupiters.bbl.
syntax error in line 714: premature end of file


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.09657-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.09657) | **A long-period transiting substellar companion in the super-Jupiters to  brown dwarfs mass regime and a prototypical warm-Jupiter detected by TESS**  |
|| M. I. Jones, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Trifonov</mark> |
|*Appeared on*| *2024-01-19*|
|*Comments*| *Accepted in A&A*|
|**Abstract**| We report on the confirmation and follow-up characterization of two long-period transiting substellar companions on low-eccentricity orbits around TIC 4672985 and TOI-2529, whose transit events were detected by the TESS space mission. Ground-based photometric and spectroscopic follow-up from different facilities, confirmed the substellar nature of TIC 4672985 b, a massive gas giant, in the transition between the super Jupiters and brown dwarfs mass regime. From the joint analysis we derived the following orbital parameters: P = 69.0480 d, Mp = 12.74 Mjup, Rp = 1.026 Rjup and e = 0.018. In addition, the RV time series revealed a significant trend at the 350 m/s/yr level, which is indicative of the presence of a massive outer companion in the system. TIC 4672985 b is a unique example of a transiting substellar companion with a mass above the deuterium-burning limit, located beyond 0.1 AU and in a nearly circular orbit. These planetary properties are difficult to reproduce from canonical planet formation and evolution models. For TOI-2529 b, we obtained the following orbital parameters: P = 64.5949 d, Mp = 2.340 Mjup, Rp = 1.030 Rjup and e = 0.021, making this object a new example of a growing population of transiting warm giant planets. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.09657.md
    + _build/html/tmp_2401.09657/./figures/mass_rad.png
    + _build/html/tmp_2401.09657/./figures/TIC4672985/LC_model.jpg
    + _build/html/tmp_2401.09657/./figures/TIC269333648/age_rad_TOI2529_v4.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cir}{\mathrm{C^{12}/C^{13}} }$
$\newcommand{\FeI}{Fe {\sc i} }$
$\newcommand{\FeII}{Fe {\sc ii} }$
$\newcommand{\feh}{[Fe/H] }$
$\newcommand{\vt}{\xi_t }$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{\kms}{km s^{-1} }$
$\newcommand{\msun}{M_\odot }$
$\newcommand{\mstar}{M_\star }$
$\newcommand{\rstar}{R_\star }$
$\newcommand{\lstar}{L_\star }$
$\newcommand{\rsun}{R_\odot }$
$\newcommand{\lsun}{L_\odot }$
$\newcommand{\loggiso}{\logg_{\text{iso}} }$
$\newcommand{\loggtri}{\logg_{\text{tri}} }$
$\newcommand{\ms}{m s^{-1} }$
$\newcommand{\mjup}{M_{\rm J} }$
$\newcommand{\rjup}{R_{\rm J} }$
$\newcommand{\vsini}{\varv sin i }$
$\newcommand{\mearth}{M_\oplus }$
$\newcommand{\zaspe}{\texttt{ZASPE} }$
$\newcommand{\SPECIES}{\texttt{SPECIES} }$
$\newcommand{\ceres}{\texttt{CERES} }$
$\newcommand{\parsec}{\texttt{PARSEC} }$
$\newcommand{\tess}{TESS }$
$\newcommand{\celerite}{\texttt{celerite}}$
$\newcommand{\mesa}{\texttt{MESA} }$
$\newcommand{\juliet}{\texttt{juliet} }$
$\newcommand{\radvel}{\texttt{radvel} }$
$\newcommand{\batman}{\texttt{batman} }$
$\newcommand{\dynesty}{\texttt{dynesty} }$
$\newcommand{\gaia}{\texttt{GAIA} }$
$\newcommand{\perone}{69.0480\substack{+0.0004 \-0.0005} }$
$\newcommand{\massone}{12.74\substack{+1.01 \-1.01} }$
$\newcommand{\radone}{1.026\substack{+0.065 \-0.067} }$
$\newcommand{\eccone}{0.018\substack{+0.004 \-0.004} }$
$\newcommand{\pertwo}{64.5949\substack{+0.0003 \-0.0003} }$
$\newcommand{\masstwo}{2.340\substack{+0.197 \-0.195} }$
$\newcommand{\radtwo}{1.030\substack{+0.050 \-0.050} }$
$\newcommand{\ecctwo}{0.021\substack{+0.024 \-0.015} }$</div>



<div id="title">

# A long-period transiting substellar companion in the super-Jupiters to brown dwarfs mass regime and a prototypical warm-Jupiter detected by $\tess$.$\thanks{ Based on observations collected at La Silla - Paranal Observatory under programs IDs 105.20GX.001, 106.212H.001, 106.21ER.001 and 108.22A8.001 and through the Chilean Telescope Time under programs IDs CN2020B-21, CN2021A-14, CN2021B-23, CN2022A-33 and CN2022B-33}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.09657-b31b1b.svg)](https://arxiv.org/abs/2401.09657)<mark>Appeared on: 2024-01-19</mark> -  _Accepted in A&A_

</div>
<div id="authors">

M. I. Jones, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Trifonov</mark>

</div>
<div id="abstract">

**Abstract:** We report on the confirmation and follow-up characterization of two long-period transiting substellar companions on low-eccentricity orbits around TIC 4672985 and TOI-2529, whose transit events were detected by the $\tess$ space mission.Ground-based photometric and spectroscopic follow-up from different facilities,confirmed the substellar nature of TIC 4672985 $b$ , a massive gas giant, in the transition between the super-Jupiters and brown-dwarfs mass regime.From the joint analysis we derived the following orbital parameters: $P$ = $\perone$ d, $M_p$ = $\massone$ $\mjup$ , $R_p$ = $\radone$ $\rjup$ and $e$ = $\eccone$ . In addition, the RV time series revealed a significant trend at the $\sim$ 350 $\ms$ yr $^{-1}$ level, which is indicative of the presence of a massive outer companion in the system.TIC 4672985 $b$ is a unique example of a transiting substellar companion with a mass above the deuterium-burning limit, located beyond 0.1 AU and in a nearly circular orbit. These planetary properties are difficult to reproduce from canonical planet formation and evolution models.For TOI-2529 $b$ , we obtained the following orbital parameters: $P$ = $\pertwo$ d, $M_p$ = $\masstwo$ $\mjup$ , $R_p$ = $\radtwo$ $\rjup$ and $e$ = $\ecctwo$ , making this object a new example of a growing population of transiting warm giant planets.

</div>

<div id="div_fig1">

<img src="tmp_2401.09657/./figures/mass_rad.png" alt="Fig12" width="100%"/>

**Figure 12. -** Mass-radius distribution of known transiting giant planets as of June 30, 2023. The open red circles and blue dots correspond to orbital periods shorter and longer than 10 $d$, respectively. The positions of TIC 4672985 $b$(black diamond) and TOI-2529 $b$(black square) are also shown. The shaded area represents the theoretical deuterium-burning limit. The solid line corresponds to a 1 Gyr old and solar metallicity ATMO2020 isochrone. Two isodensity curves for 1 and 10 [gr cm$^{-3}$] are also plotted (dashed left and right lines, respectively).   (*fig:mass_rad*)

</div>
<div id="div_fig2">

<img src="tmp_2401.09657/./figures/TIC4672985/LC_model.jpg" alt="Fig4" width="100%"/>

**Figure 4. -** Normalized photometric observations of TIC 4672985 during the transit. The data were obtained in $\tess$ sector 4 (upper panel) and from ground-based telescopes (CALLISTO and OM-ES; middle and lower panels, respectively). The solid lines correspond to the transit models.  (*fig:LC_model_TIC4672985*)

</div>
<div id="div_fig3">

<img src="tmp_2401.09657/./figures/TIC269333648/age_rad_TOI2529_v4.png" alt="Fig15" width="100%"/>

**Figure 15. -** Position of TOI-2529 $b$ in the age-radius diagram (black dot). Planet evolutionary models with different envelope composition and insolation level over-plotted. For comparison, a simple model of Jupiter is also shown.
 (*fig:age_rad_TOI2529*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.09657"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

355  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

1  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
